In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [ ]:
global labels
global features
labels = [f"label_{i+1}" for i in range(4)]
features = [f'feature_{i+1}' for i in range(768)]
label_1 = labels[0]
label_2 = labels[1]
label_3 = labels[2]
label_4 = labels[3]

In [ ]:
train = pd.read_csv("/kaggle/input/ml-project/train.csv")
valid = pd.read_csv("/kaggle/input/ml-project/valid.csv")
test = pd.read_csv("/kaggle/input/ml-project/test.csv")

In [ ]:
def data_preprocess(train_df, valid_df, test_df, label):
    scaler = StandardScaler()

    # Apply feature scaling to training data
    X_train = pd.DataFrame(scaler.fit_transform(train_df.drop(labels, axis=1)), columns=features)
    y_train = train_df[label]

    # Apply feature scaling to validation data
    X_valid = pd.DataFrame(scaler.transform(valid_df.drop(labels, axis=1)), columns=features)
    y_valid = valid_df[label]

    # Apply feature scaling to test data
    X_test = pd.DataFrame(scaler.transform(test_df.drop("ID", axis=1)), columns=features)
    return X_train, y_train, X_valid, y_valid, X_test

In [ ]:
def feature_engineering_with_pca(X_train, X_valid, X_test, n_components):

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components)
    
    # Create a new DataFrame with the PCA-transformed features
    X_train_pca = pca.fit_transform(X_train)  
    X_train_df_pca = pd.DataFrame(X_train_pca)
    X_valid_df_pca = pd.DataFrame(pca.transform(X_valid))
    X_test_df_pca = pd.DataFrame(pca.transform(X_test))
    print("Shape for label4 train set:", X_train_df_pca.shape)
    print("Shape for label4 validation set:", X_valid_df_pca.shape)
    print("Shape for label4 validation set:", X_test_df_pca.shape)

    return X_train_df_pca, X_valid_df_pca, X_test_df_pca

In [ ]:
def feature_engineering_with_select_kbest(X_train, y_train, X_valid, X_test, n):
    selector = SelectKBest(f_classif, k=n)
    X_train_df = selector.fit_transform(X_train, y_train)
    X_valid_df = selector.transform(X_valid)
    X_test_df = selector.transform(X_test)
    print("Shape for label4 train set:", X_train_df_pca.shape)
    print("Shape for label4 validation set:", X_valid_df_pca.shape)
    print("Shape for label4 validation set:", X_test_df_pca.shape)
    return X_train_df, X_valid_df, X_test_df

In [ ]:
def build_knn_model(X_train, y_train, n_neighbors):
    # Create and train your KNN classifier model
    # You can perform hyperparameter tuning by changing the value of 'n_neighbors'
    knn = KNeighborsClassifier(n_neighbors)  
    knn.fit(X_train, y_train)
    return knn

In [ ]:
def build_svc_model(X_train, y_train):
    svc_model = SVC(kernel='linear', gamma='scale')
    svc_model.fit(X_train, y_train)
    return svc_model

In [ ]:
def evaluate(y_valid, y_pred):
    print(f"Accuracy: {accuracy_score(y_valid, y_pred)}")
    print(f"F1 Score: {f1_score(y_valid, y_pred, average='weighted')}")
    print(f"Precision: {precision_score(y_valid, y_pred, average='weighted')}")
    print(f"Recall: {recall_score(y_valid, y_pred, average='weighted')}")

# Label 04

In [ ]:
X_train_4, y_train_4, X_valid_4, y_valid_4, X_test_4 = data_preprocess(train, valid, test, label_4)
y_train_4.info()

In [ ]:
plt.hist(y_train_4, bins=20, edgecolor='k')
plt.xlabel(label_4)
plt.ylabel('Count')
plt.title(f'Distribution of {label_4}')
plt.show()

# Before doing anything

In [ ]:
# Build a KNN model
model_4_before = build_knn_model(X_train_4, y_train_4, 5)

# Evaluate the model on the validation set
y_pred_4_before = model_4_before.predict(X_valid_4)

# Calculate evaluation metrics 
evaluate(y_valid_4, y_pred_4_before)

In [ ]:
resampler = RandomOverSampler(sampling_strategy='auto')
X_train_4_resampled, y_train_4_resampled = resampler.fit_resample(X_train_4, y_train_4)

In [ ]:
plt.hist(y_train_4_resampled, bins=20, edgecolor='k')
plt.xlabel(label_4)
plt.ylabel('Frequency')
plt.title(f'Distribution of {label_4}')
plt.show()

In [ ]:
# Build a KNN model after oversampling
model_4_oversampling = build_knn_model(X_train_4_resampled, y_train_4_resampled, 5)

# Evaluate the model on the validation set
y_pred_4_oversampling = model_4_oversampling.predict(X_valid_4)

# Calculate evaluation metrics 
evaluate(y_valid_4, y_pred_4_oversampling)

In [ ]:
# Build a SVC model 
model_4_oversampling_svc = build_svc_model(X_train_4_resampled, y_train_4_resampled)

# Evaluate the model on the validation set
y_pred_4_oversampling_svc = model_4_oversampling_svc.predict(X_valid_4)

# Calculate evaluation metrics 
evaluate(y_valid_4, y_pred_4_oversampling_svc)

# Feature reduction using PCA

In [ ]:
selector = SelectKBest(f_classif, k=350)
X_train_4_new = selector.fit_transform(X_train_4_resampled, y_train_4_resampled)
X_valid_4_new = selector.transform(X_valid_4)
X_test_4_new = selector.transform(X_test_4)

In [ ]:
X_train_4_pca, X_valid_4_pca, X_test_4_pca = feature_engineering_with_pca(X_train_4_new, X_valid_4_new, X_test_4_new, n_components=0.95)

In [ ]:
# Build a KNN model 
model_4_knn = build_knn_model(X_train_4_pca, y_train_4_resampled, 5)

# Evaluate the model on the validation set
y_pred_4_knn = model_4_knn.predict(X_valid_4_pca)

# Calculate evaluation metrics 
evaluate(y_valid_4, y_pred_4_knn)

# Hyperparameter Tuning for label 4

In [ ]:
param_grid_4 = {
    'C': [0.1, 1, 10, 100],         
    'kernel': ['linear', 'rbf', 'poly'] 
}

model_4_svc = SVC()

grid_search_4 = GridSearchCV(estimator=model_4_svc, param_grid=param_grid_4, cv=5, verbose=3, n_jobs=-1, scoring='accuracy')

grid_search_4.fit(X_train_4_pca, y_train_4_resampled)

best_params_4 = grid_search_4.best_params_
best_model_4 = grid_search_4.best_estimator_

valid_accuracy_4 = best_model_4.score(X_valid_4_pca, y_valid_4)

# Print the best hyperparameters and test accuracy
print("Best Hyperparameters:", best_params_4)
print("Test Accuracy:", valid_accuracy_4)

# Predicting label 4 